We periodically cache information about taxa of interest from the USFWS Threatened and Endangered Species System (TESS) in the Taxonomic Information Registry. This information is displayed in places like the State Wildlife Action Plan (SWAP) online app and the National Biogeographic Map.

TESS data are stored as key/value pairs in an hstore column of the "tir2" data store in the experimental GC2 platform we are working with. This notebook shows a couple of methods for retrieving TESS information via the GC2 API for use in web applications.

There is a TESS record for every taxon that is registered in the TIR. When no information from TESS was available with the query (based on ITIS TSN), there will be a result=>none key/value and a cacheDate, incidating the date/time the query was run.

The following queries, based on the SQL API, are probably the best options with hstore values. The data have also been piped to ElasticSearch in the GC2 instance, but they will come back as strings that need to be parsed instead of simple JSON. Note that there should only be a single result coming back for any given TSN or any originally submitted scientific name from the SGCN. However, there is not an absolute requirement that only a single record exist in the database for any of these key identifying features. You might harden your code by checking to make sure that there is only a single record and dealing with any duplicates.

In [1]:
import requests
from IPython.display import display

In [5]:
# Query based on ITIS tsn
tsn = 201922

q = "SELECT to_json(tess) as tess FROM tir.tir2 \
    WHERE tess->'tsn' = '"+str(tsn)+"'"

r = requests.get("https://gc2.mapcentia.com/api/v1/sql/bcb?q="+q).json()

display (r["features"][0]["properties"]["tess"])

{'CommonName': 'Pahranagat roundtail chub',
 'ListingDate': '1970-10-13',
 'PopulationDescription': 'Wherever found',
 'SpeciesCode': 'E00R',
 'Status': 'E',
 'StatusText': 'Endangered',
 'cacheDate': '2017-05-07T16:38:34.510363',
 'entityId': '226',
 'result': 'success',
 'tsn': '201922'}

In [6]:
# Query based on a submitted SGCN scientific name
scientificname = "Solanum nelsonii"

q = "SELECT to_json(tess) as tess FROM tir.tir2 \
    WHERE registration->'SGCN_ScientificName_Submitted' = '"+scientificname+"'"

r = requests.get("https://gc2.mapcentia.com/api/v1/sql/bcb?q="+q).json()

display (r["features"][0]["properties"]["tess"])

{'CommonName': 'Popolo',
 'ListingDate': '2016-10-31',
 'PopulationDescription': 'Wherever found',
 'SpeciesCode': 'Q21R',
 'Status': 'E',
 'StatusText': 'Endangered',
 'cacheDate': '2017-05-07T16:38:54.242871',
 'entityId': '6870',
 'result': 'success',
 'tsn': '30483'}